# This is ML Rewiew rating model, so let's go!

In [ ]:
import os
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from transformers import BertTokenizer

### Set up data reader

In [ ]:
DATA_DIR = Path('../DataSets/dataset-LONG-films')

labels_map = {
    'negative': 0,
    'neutral': 1,
    'positive': 2
}


def load_data_from_folders(base_path, mapping):
    data = []
    
    # go for each folder
    for folder_name, label_id in mapping.items():
        folder_path = base_path / folder_name
        
        # list of all txt files in there
        files = list(folder_path.glob('*.txt'))
        
        print(f"Loading {folder_name}: found {len(files)} files")
        
        # Read every file
        for file_path in tqdm(files, desc=folder_name):
            try:
                # Skip unreadable files
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    text = f.read()
                    # Add [text, label] in list
                    data.append({'text': text, 'label': label_id})
            except Exception as e:
                print(f"Ошибка чтения файла {file_path}: {e}")
                
    return pd.DataFrame(data)

##### Start it:

In [ ]:
df_of_data_in_folders = load_data_from_folders(DATA_DIR, labels_map)
print(f"\nLoaded rows: {len(df_of_data_in_folders)}")

##### Save it:

In [ ]:
df_of_data_in_folders.to_csv('data_in_folders.csv')

##### Load data:

In [6]:
df_of_data_in_folders = pd.read_csv('data_in_folders.csv')
print(f"\nLoaded rows: {len(df_of_data_in_folders)}")


Loaded rows: 131669


##### Split data:

In [7]:
train_df, test_df = train_test_split(df_of_data_in_folders, test_size=0.2, random_state=42, stratify=df_of_data_in_folders['label'])

print(f"Train data: {len(train_df)}")
print(f"test data: {len(test_df)}")

Train data: 105335
test data: 26334


##### Add dataloader + built-in tokenizer 

In [ ]:
class BertTextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=256):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get text and label
        row = self.data.iloc[idx]
        text = str(row['text'])
        label = row['label']

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,  # Add [CLS], [SEP]
            max_length=self.max_len,
            padding='max_length',
            truncation=True,          # Cut if bigger than max_length
            return_attention_mask=True,
            return_tensors='pt',
        )

        # Return label and ID
        # .flatten() cut extra dimentions
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


# Multi language tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# set up datasets
train_dataset = BertTextDataset(train_df, tokenizer)
test_dataset = BertTextDataset(test_df, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

   Unnamed: 0                                               text  label
0           0  Что же написать о фильме? По прошествии 2 лет ...      0
1           1  Очень не хотелось писать рецензию по сериалам,...      0
2           2  Ох, не люблю я день святого Валентина! Отнюдь ...      0
3           3  Вам что-то говорит имя Александр Уитт? Мне нет...      0
4           4  После первых двух частей чего-то от третьей я ...      0
